libraries

In [51]:
# Cell 0 — install what's needed
!pip install requests jsonschema --quiet

Set your API key safely

In [ ]:
# Cell 1 — enter your Groq API key safely (it won't print)
from getpass import getpass
import os

GROQ_API_KEY = getpass("Paste your Groq API key (won't show): ")
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

# Groq's OpenAI-compatible base URL
GROQ_BASE = "https://api.groq.com/openai/v1"

# Quick check (do not print key)
print("API key loaded into session. GROQ_BASE is set.")


Small helper to call Groq with requests

In [ ]:
# Cell 2 — helper for calling the Groq OpenAI-compatible chat/completions endpoint
import requests, json, time

HEADERS = {
    "Authorization": f"Bearer {os.environ.get('GROQ_API_KEY')}",
    "Content-Type": "application/json"
}

def groq_chat_request(messages, model="llama3-8b-8192", functions=None, function_call=None, temperature=0.2, max_tokens=512):
    payload = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    if functions is not None:
        payload["functions"] = functions
    if function_call is not None:
        payload["function_call"] = function_call

    resp = requests.post(f"{GROQ_BASE}/chat/completions", headers=HEADERS, json=payload)
    resp.raise_for_status()
    return resp.json()

ConversationHistory class

In [ ]:
# Cell 3 — conversation history manager with truncation and summary replacement
from typing import List, Dict, Optional

class ConversationHistory:
    def __init__(self):
        self.history: List[Dict[str,str]] = []   # list of {role, content}
        self.run_count = 0

    def add_message(self, role: str, content: str):
        self.history.append({"role": role, "content": content})

    def last_n_turns(self, n: int) -> List[Dict[str,str]]:
        # Return last n messages (user+assistant messages counted individually)
        return self.history[-n:] if n > 0 else []

    def truncate_by_chars(self, max_chars: int) -> List[Dict[str,str]]:
        # Keep most recent messages until the char budget is filled
        total = 0
        selected = []
        for msg in reversed(self.history):
            l = len(msg['content'])
            if total + l > max_chars:
                break
            selected.insert(0, msg)
            total += l
        return selected

    def replace_with_summary(self, summary_text: str, upto_index: Optional[int] = None):
        # Replace history up to upto_index (exclusive) with a single system summary
        if upto_index is None:
            upto_index = len(self.history)
        remaining = self.history[upto_index:]
        self.history = [{"role":"system", "content": "Conversation summary:\n" + summary_text}] + remaining

    def show(self):
        print("---- Conversation history ----")
        for i, m in enumerate(self.history):
            role = m['role']
            content = m['content']
            print(f"{i:02d} | {role}: {content}")
        print("-----------------------------")


LLM summarization & fallback

In [ ]:
# Cell 4 — LLM summarization wrapper plus a tiny fallback
def build_summary_prompt(messages: List[Dict[str,str]]) -> str:
    lines = []
    for m in messages:
        lines.append(f"[{m['role']}] {m['content']}")
    messages_text = "\n".join(lines)
    prompt = (
        "You are a helpful assistant. Summarize the conversation below in 2-3 short sentences, "
        "mentioning main requests and any user info (name, email, phone, location, age) if present.\n\n"
        f"{messages_text}\n\nSummary:"
    )
    return prompt

def llm_summarize(messages_slice: List[Dict[str,str]], model="llama3-8b-8192"):
    # Build a single user prompt message sent to model
    prompt = build_summary_prompt(messages_slice)
    payload_messages = [{"role":"user", "content": prompt}]
    try:
        resp = groq_chat_request(payload_messages, model=model, temperature=0.0, max_tokens=256)
        choice = resp['choices'][0]
        summary_text = choice['message']['content'].strip()
        return summary_text
    except Exception as e:
        print("LLM summarization failed:", e)
        # fallback: extract last 2 user messages
        user_texts = [m['content'] for m in messages_slice if m['role']=='user']
        fallback = "Recent user notes: " + " | ".join(user_texts[-2:])
        return fallback

Periodic summarization runner

In [ ]:
# Cell 5 — function to add a user turn and periodically summarize after every k runs
def run_user_turn(conv: ConversationHistory, user_text: str, assistant_text_fn=None, k: int=3, truncation_mode: str='turns', truncation_value: int=6):
    """
    conv: ConversationHistory
    user_text: new user message (string)
    assistant_text_fn: optional function that takes user_text and returns assistant response string
                       if None, assistant reply will be simple echo.
    k: summarize every k runs (periodic summarization)
    truncation_mode: 'turns' or 'chars'
    truncation_value: if turns -> last n messages used for summary; if chars -> max chars used
    """
    conv.run_count += 1
    conv.add_message('user', user_text)

    # create assistant reply (here: simple echo, or call the model if you like)
    if assistant_text_fn is None:
        assistant_reply = "Assistant: got your message - " + (user_text[:120])
    else:
        assistant_reply = assistant_text_fn(user_text)
    conv.add_message('assistant', assistant_reply)

    # Periodic summarization
    if conv.run_count % k == 0:
        # choose slice for summarization
        if truncation_mode == 'turns':
            slice_msgs = conv.last_n_turns(truncation_value)
        else:
            slice_msgs = conv.truncate_by_chars(truncation_value)
        print(f">>> Running summarization at run {conv.run_count} (k={k}) - using {truncation_mode}={truncation_value}")
        summary = llm_summarize(slice_msgs)
        conv.replace_with_summary(summary)   # replace older content with the summary

    return assistant_reply


Task 1 — feed multiple samples & show different truncations

In [ ]:
# Cell 6 — Demo for Task 1
conv = ConversationHistory()

samples = [
    "Hi, I'm Anu. My email is anu@example.com and I live in Pune.",
    "I would like to upgrade to premium subscription.",
    "Also, can you export my chat history and billing info?",
    "I am 31 years old and my phone number is +919876543210.",
    "What is the refund policy if I cancel within 7 days?",
    "Can you provide invoice for last month?",
    "Thanks, I need it by next Monday."
]

# run through messages, summarizing every 3 runs (k=3), using last 6 messages for the summary
for i, msg in enumerate(samples, start=1):
    reply = run_user_turn(conv, msg, k=3, truncation_mode='turns', truncation_value=6)
    print(f"Run {i} assistant reply: {reply}")
    conv.show()
    print("\n---\n")

# Now try again using char truncation for comparison
print("Now demo truncation by chars (max 180 chars for summary slice).")
conv2 = ConversationHistory()
for i, msg in enumerate(samples, start=1):
    _ = run_user_turn(conv2, msg, k=3, truncation_mode='chars', truncation_value=180)
conv2.show()

Task 2 — JSON schema + function-calling definition

In [ ]:
# Cell 7 — JSON schema and function definition (OpenAI-style) for extracting 5 fields
CONTACT_SCHEMA = {
    "type":"object",
    "properties":{
        "name": {"type":"string"},
        "email": {"type":["string","null"]},
        "phone": {"type":["string","null"]},
        "location": {"type":["string","null"]},
        "age": {"type":["integer","null"]}
    },
    "required": ["name"]
}

# Function spec to pass to the model
FUNCTIONS = [
    {
        "name": "extract_contact",
        "description": "Extract contact fields from a chat message",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type":"string", "description":"Full name if present"},
                "email": {"type":"string", "description":"Email address or null"},
                "phone": {"type":"string", "description":"Phone number or null"},
                "location": {"type":"string", "description":"City or location or null"},
                "age": {"type":"integer", "description":"Age in years or null"}
            },
            "required": ["name"]
        }
    }
]


Helper to call function-calling and parse results

In [ ]:
# Cell 8 — function-calling helper that asks model to run extract_contact and returns parsed JSON
def extract_contact_via_model(sample_chat: str, model="llama3-8b-8192"):
    messages = [
        {"role":"system", "content":"You are a JSON parser. Extract user info into the function arguments exactly."},
        {"role":"user", "content": sample_chat}
    ]
    try:
        resp = groq_chat_request(messages, model=model, functions=FUNCTIONS, function_call={"name":"extract_contact"}, temperature=0.0, max_tokens=256)
        choice = resp['choices'][0]
        # The model should return function_call arguments
        if 'message' in choice and 'function_call' in choice['message']:
            args_text = choice['message']['function_call']['arguments']
            parsed = json.loads(args_text)
            return parsed, resp
        # fallback: try parsing content
        if 'message' in choice and 'content' in choice['message']:
            try:
                parsed = json.loads(choice['message']['content'])
                return parsed, resp
            except Exception:
                return None, resp
    except Exception as e:
        print("Model call failed:", e)
        return None, {}

Task 2 — parse 3 sample chats and validate

In [ ]:
# Cell 9 — run extraction on 3 sample chats and validate using jsonschema
from jsonschema import validate, ValidationError
samples = [
    "Hello, I'm Priya R. My email is priya.r@example.com and I live in Bangalore. I'm 27.",
    "Hi, it's Raj. Phone: +919876543210. I don't have an email listed.",
    "Name: John Doe; Age: 45; Location: New York; Contact: john.doe@example.com, +1-415-555-0123"
]

results = []
for s in samples:
    parsed, raw = extract_contact_via_model(s)
    print("Sample:", s)
    print("Parsed (raw):", parsed)
    if parsed:
        try:
            # Ensure age is integer or null
            if parsed.get("age") is not None and isinstance(parsed.get("age"), str) and parsed.get("age").isdigit():
                parsed["age"] = int(parsed["age"])
            validate(instance=parsed, schema=CONTACT_SCHEMA)
            print("Validation: OK")
        except ValidationError as ve:
            print("Validation error:", ve.message)
    else:
        print("No parsed JSON returned.")
    print("\n---\n")
    results.append({"sample": s, "parsed": parsed})

# Show final results list
print("All results:", results)

Save notebook and push to GitHub (safe workflow)

In [ ]:
# 1. Check the file exists
!ls -lh /content

# 2. Go to content folder
%cd /content

# 3. Configure git (only first time, replace with your details)
!git config --global user.email "you@example.com"
!git config --global user.name "Your Name"

# 4. Initialize git (safe even if already initialized)
!git init

# 5. Reset and add remote (to avoid 'remote origin already exists' error)
!git remote remove origin || true
!git remote add origin https://github.com/manju1vr/conversation-management-groq.git

# 6. Add your notebook
!git add "conversation_management.ipynb"

# 7. Commit it
!git commit -m "Add conversation management notebook"

# 8. Make sure branch is main
!git branch -M main

# 9. Push to GitHub
!git push -u origin main
